# LEGO Nowosci Scraper 2026

Nowoczesny scraper nowosci LEGO z uzyciem Playwright.
Dziala na Google Colab i lokalnie.

In [ ]:
# Instalacja - uruchom raz
!pip install -q playwright pandas nest_asyncio matplotlib
!playwright install chromium
!playwright install-deps chromium
print("Instalacja zakonczona!")

In [ ]:
import asyncio
import pandas as pd
import re
from datetime import datetime
from playwright.async_api import async_playwright

async def scrape_lego_nowosci():
    """Scrapuje nowosci LEGO - wersja 2026."""
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            viewport={'width': 1920, 'height': 1080},
            locale='pl-PL',
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        )
        page = await context.new_page()
        
        print("Otwieram strone LEGO...")
        await page.goto('https://www.lego.com/pl-pl/categories/new-sets-and-products', timeout=60000)
        await page.wait_for_timeout(5000)
        
        # Zamknij popupy
        for selector in ['#age-gate-grown-up-cta', 'button[data-test="age-gate-grown-up-cta"]', 'button:has-text("18+")']:
            try:
                btn = page.locator(selector)
                if await btn.count() > 0:
                    await btn.first.click(timeout=3000)
                    await page.wait_for_timeout(1000)
                    break
            except:
                pass
        
        # Cookies
        for selector in ['button[data-test="cookie-accept-all"]', 'button:has-text("Zaakceptuj wszystkie")']:
            try:
                btn = page.locator(selector)
                if await btn.count() > 0:
                    await btn.first.click(timeout=3000)
                    await page.wait_for_timeout(1000)
                    break
            except:
                pass
        
        # Scrolluj
        print("Laduje produkty...")
        for i in range(10):
            await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
            await page.wait_for_timeout(2000)
            print(f"  Scroll {i+1}/10")
        
        await page.wait_for_timeout(3000)
        
        # Pobierz dane przez JavaScript - szybko i niezawodnie
        print("Pobieram dane produktow...")
        products = await page.evaluate('''() => {
            const products = [];
            
            // Znajdz wszystkie linki do produktow
            const links = document.querySelectorAll('a[href*="/product/"]');
            const seen = new Set();
            
            links.forEach(link => {
                const href = link.href;
                if (seen.has(href)) return;
                seen.add(href);
                
                // Znajdz kontener produktu (parent elements)
                let container = link.closest('li') || link.closest('article') || link.closest('[data-test*="product"]') || link.parentElement?.parentElement?.parentElement;
                if (!container) return;
                
                const text = container.innerText || '';
                const lines = text.split('\\n').map(l => l.trim()).filter(l => l);
                
                // Parsuj dane
                let nazwa = '';
                let cena = '';
                let wiek = '';
                let elementy = '';
                let ocena = '';
                
                for (const line of lines) {
                    // Cena - zawiera "zł"
                    if (line.includes('zł') && !cena) {
                        cena = line;
                    }
                    // Wiek - format "X+"
                    else if (/^\\d+\\+$/.test(line) && !wiek) {
                        wiek = line;
                    }
                    // Elementy - same cyfry (liczba elementow)
                    else if (/^\\d{2,}$/.test(line) && !elementy) {
                        elementy = line;
                    }
                    // Ocena - format "X.X" lub "X,X"
                    else if (/^\\d[.,]\\d$/.test(line) && !ocena) {
                        ocena = line;
                    }
                    // Nazwa - dlugi tekst bez specjalnych znakow
                    else if (line.length > 3 && !line.includes('zł') && !/^\\d+\\+?$/.test(line) && !nazwa) {
                        // Sprawdz czy to nie jest przycisk
                        if (!line.toLowerCase().includes('koszyk') && 
                            !line.toLowerCase().includes('dostępn') &&
                            !line.toLowerCase().includes('przedsprze') &&
                            !line.toLowerCase().includes('nowość') &&
                            !line.toLowerCase().includes('wkrótce')) {
                            nazwa = line;
                        }
                    }
                }
                
                // Dostepnosc
                let dostepnosc = 'Nieznana';
                const textLower = text.toLowerCase();
                if (textLower.includes('dodaj do koszyka')) dostepnosc = 'Dostepny';
                else if (textLower.includes('niedostępn')) dostepnosc = 'Niedostepny';
                else if (textLower.includes('przedsprzedaż')) dostepnosc = 'Przedsprzedaz';
                else if (textLower.includes('wkrótce')) dostepnosc = 'Wkrotce dostepne';
                
                if (nazwa) {
                    products.push({
                        nazwa: nazwa,
                        cena: cena,
                        wiek: wiek,
                        elementy: elementy,
                        ocena: ocena,
                        dostepnosc: dostepnosc,
                        url: href
                    });
                }
            });
            
            return products;
        }''')
        
        await browser.close()
        
        print(f"Znaleziono {len(products)} produktow")
        
        # Przetworz dane
        for p in products:
            # Cena numeryczna
            if p['cena']:
                match = re.search(r'([\d\s]+[,.]?\d*)', p['cena'].replace(' ', '').replace('\xa0', ''))
                if match:
                    try:
                        p['cena_zl'] = float(match.group(1).replace(',', '.').replace(' ', ''))
                    except:
                        p['cena_zl'] = None
                else:
                    p['cena_zl'] = None
            else:
                p['cena_zl'] = None
            
            # Elementy jako int
            if p['elementy']:
                try:
                    p['elementy'] = int(p['elementy'])
                except:
                    p['elementy'] = None
            else:
                p['elementy'] = None
                
            # Ocena jako float
            if p['ocena']:
                try:
                    p['ocena'] = float(p['ocena'].replace(',', '.'))
                except:
                    p['ocena'] = None
            else:
                p['ocena'] = None
        
        return products

print("Gotowe! Uruchom nastepna komorke.")

In [ ]:
# URUCHOM SCRAPER
print("="*50)
print("LEGO NOWOSCI SCRAPER 2026")
print("="*50)

# Dla Jupyter/Colab
import nest_asyncio
nest_asyncio.apply()

produkty = asyncio.get_event_loop().run_until_complete(scrape_lego_nowosci())

print(f"\n{'='*50}")
print(f"POBRANO: {len(produkty)} produktow")
print(f"{'='*50}")

In [ ]:
# Utworz DataFrame
df = pd.DataFrame(produkty)
df['data_pobrania'] = datetime.now().strftime('%Y-%m-%d %H:%M')

print(f"Produktow: {len(df)}")
df

In [ ]:
# STATYSTYKI
print("\n" + "="*40)
print("STATYSTYKI")
print("="*40)

if df['cena_zl'].notna().any():
    print(f"\nCENY:")
    print(f"  Najtanszy: {df['cena_zl'].min():.2f} zl")
    print(f"  Najdrozszy: {df['cena_zl'].max():.2f} zl") 
    print(f"  Srednia: {df['cena_zl'].mean():.2f} zl")

if df['elementy'].notna().any():
    print(f"\nELEMENTY:")
    print(f"  Min: {df['elementy'].min()}")
    print(f"  Max: {df['elementy'].max()}")

print(f"\nDOSTEPNOSC:")
print(df['dostepnosc'].value_counts().to_string())

In [ ]:
# ZAPISZ DO CSV
plik = f"lego_nowosci_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
df.to_csv(plik, index=False, encoding='utf-8-sig')
print(f"Zapisano: {plik}")

# Pobierz na Colab
try:
    from google.colab import files
    files.download(plik)
except:
    pass

In [ ]:
# TOP 10 NAJDROZSZYCH
print("\nTOP 10 NAJDROZSZYCH:")
print("-"*40)
top10 = df.nlargest(10, 'cena_zl')[['nazwa', 'cena', 'elementy']]
for i, row in top10.iterrows():
    print(f"{row['nazwa'][:40]:40} | {row['cena']:>12} | {row['elementy']} elem.")

In [ ]:
# WYKRES CEN
import matplotlib.pyplot as plt

if df['cena_zl'].notna().any():
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram cen
    axes[0].hist(df['cena_zl'].dropna(), bins=25, color='#FFD700', edgecolor='black')
    axes[0].set_xlabel('Cena (zl)')
    axes[0].set_ylabel('Liczba produktow')
    axes[0].set_title('Rozklad cen nowosci LEGO')
    axes[0].grid(True, alpha=0.3)
    
    # Dostepnosc
    colors = ['#4CAF50', '#f44336', '#FF9800', '#2196F3', '#9E9E9E']
    df['dostepnosc'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.1f%%', colors=colors[:len(df['dostepnosc'].unique())])
    axes[1].set_title('Dostepnosc produktow')
    axes[1].set_ylabel('')
    
    plt.tight_layout()
    plt.show()